# Engineer
Here, we will engineer our features as follows:
* Convert numeric data to categorical
    * MoSold
    * MSSubClass
* Convert categorical data with orderings (likert scale type data) into ordinal data
E.g.,  `GarageQual`: NoGarage->0, Po->1, Fa->2, TA->3, Gd->4, Ex->5
* Create new features
    * Derived
    E.g., `NumFloors` in a property
    * Indicators (booleans)
    E.g., `IsPUD` whether a property is in a PUD
* Collapse number of categories for categorical features on a case-by-case basis
E.g., `OverallQual` has scale 1-10. 1-3 are simplified to "bad", 4-6 are "normal", 7-10 are "good"
* Dummify categorical features

In [264]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [265]:
import pandas as pd
import data_dict

In [266]:
df = pd.read_csv("../data/cleaned.csv")
df.drop('Unnamed: 0', inplace=True, axis=1)

## Convert numerical data to categorical
`YrSold`, `MoSold`, and `MSSubClass` are categorical data but are encoded as integers in the dataset

In [267]:
# NOTE: We will treat MoSold and YrSold as categorical because there are only 12 (Jan->Dec) and 5 values (2006->2010).
# This might not be the case if we had many years of sales data in our dataset
df['YrSold'] = df['YrSold'].astype(str)
df['MoSold'] = df.MoSold.map(data_dict.convert_mosold)
df['MSSubClass'] = df['MSSubClass'].astype(str)

## Convert categorical to ordinal

In [268]:
# Overall features
df['OverallQual'] = df.OverallQual.map(data_dict.convert_overallqual)
df['OverallCond'] = df.OverallCond.map(data_dict.convert_overallcond)

In [269]:
# Exterior features
df['ExterQual'] = df.ExterQual.map(data_dict.convert_exterqual)
df['ExterCond'] = df.ExterCond.map(data_dict.convert_extercond)

In [270]:
# Basement features
df['BsmtQual'] = df.BsmtQual.map(data_dict.convert_bsmtqual)
df['BsmtCond'] = df.BsmtCond.map(data_dict.convert_bsmtcond)
df['BsmtExposure'] = df.BsmtExposure.map(data_dict.convert_bsmtexposure)
df['BsmtFinType1'] = df.BsmtFinType1.map(data_dict.convert_bsmtfintype)
df['BsmtFinType2'] = df.BsmtFinType2.map(data_dict.convert_bsmtfintype)

In [271]:
# Home Interior features
df['Functional'] = df.Functional.map(data_dict.convert_functional)
df['FireplaceQu'] = df.FireplaceQu.map(data_dict.convert_fireplacequ)
df['HeatingQC'] = df.HeatingQC.map(data_dict.convert_heatingqc)
df['KitchenQual'] = df.KitchenQual.map(data_dict.convert_kitchenqual)

In [272]:
# Land features
df['LandSlope'] = df.LandSlope.map(data_dict.convert_landslope)
df['LotShape'] = df.LotShape.map(data_dict.convert_lotshape)

In [273]:
# Garage features
df['GarageCond'] = df.GarageCond.map(data_dict.convert_garagecond)
df['GarageQual'] = df.GarageQual.map(data_dict.convert_garagequal)

In [274]:
# Road features
df['Street'] = df.Street.map(data_dict.convert_street)
df['PavedDrive'] = df.PavedDrive.map(data_dict.convert_paveddrive)
df['Alley'] = df.Alley.map(data_dict.convert_alley)

In [275]:
# Other features
df['Utilities'] = df.Utilities.map(data_dict.convert_utilities)
df['PoolQC'] = df.PoolQC.map(data_dict.convert_poolqc)

## New Features

#### Indicators

In [276]:
# See whether a property is in a PUD from the dwelling type
df['IsPUD'] = df.MSSubClass.map(data_dict.get_pud_indicator)

In [277]:
# TODO LotShape isRegular

#### Other new features

In [278]:
# Get number of floors for the property from the dwelling type
df['NumFloors'] = df.MSSubClass.map(data_dict.get_num_floors)

In [279]:
df['BsmtAllBaths'] = df['BsmtFullBath'] + df['BsmtHalfBath']*.25
df['AbvGrdBaths'] = df['FullBath'] + df['HalfBath']*.25

## Collapse Categorical Features
Use smaller scales for Likert scale-type categorical features

In [280]:
df['Collapse_MSSubClass'] = df.MSSubClass.map(data_dict.collapse_mssubclass)
# TODO ask team others: (*Qual, *Cond)->*QualCond, Year->Age, IsNew,

## Features by Type of Data

In [281]:
guid = ['PID']
target = ['SalePrice']

ordinal = ['ExterQual', 'ExterCond', 'OverallCond', 'OverallQual',
           'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
           'Functional', 'FireplaceQu', 'HeatingQC', 'KitchenQual', 'LandSlope', 'LotShape',
           'GarageCond', 'GarageQual', 'Street', 'PavedDrive', 'Alley', 'Utilities', 'PoolQC']

# TODO create more based on R^2. We want to create features only if we will use them
collapse_categorical = ['Collapse_MSSubClass']
categorical = ['MSSubClass', 'MSZoning',
               'LandContour', 'LotConfig', 'Neighborhood',
               'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
               'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating',
               'CentralAir', 'Electrical', 'GarageType', 'GarageFinish', 'Fence', 'MiscFeature',
               'MoSold', 'YrSold',
               'SaleType', 'SaleCondition']
categorical = categorical + collapse_categorical

new_numerical = ['NumFloors', 'BsmtAllBaths', 'AbvGrdBaths']
numerical = ['GrLivArea', 'LotArea', 'LotFrontage', 'MasVnrArea',
             'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
             '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
             'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
             'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea',
             'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
             'YearBuilt', 'YearRemodAdd',
             'MiscVal']
numerical = numerical + new_numerical

# TODO create more based on R^2. We want to create features only if we will use them
booleans = ['IsPUD']

Check that we organized every feature

In [282]:
# Test that we have all features labeled by type of data
features = ordinal + categorical + numerical + booleans
print(f"{len(df.columns)} #columns == {len(guid + target + features)} #features+PID+SalePrice")

86 #columns == 86 #features+PID+SalePrice


In [283]:
# Manually check that these are the same
pd.concat([pd.Series(sorted(features+guid+target)),
           pd.Series(sorted(df.columns))], axis=1)

,0,1
0,1stFlrSF,1stFlrSF
1,2ndFlrSF,2ndFlrSF
2,3SsnPorch,3SsnPorch
3,AbvGrdBaths,AbvGrdBaths
4,Alley,Alley
...,...,...
81,Utilities,Utilities
82,WoodDeckSF,WoodDeckSF
83,YearBuilt,YearBuilt
84,YearRemodAdd,YearRemodAdd


Save output

In [284]:
df.to_csv("../data/engineered.csv")